In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import dask.dataframe as dd
import dask.multiprocessing
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.metrics import accuracy_score
import pickle
import lightgbm as lgb
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier
import xgboost as xgb

# Load Data
## 2.1. Opcodes

In [2]:
#load data
df_opcodes = pd.read_csv("../data/interim/opcodes.csv")

In [3]:
df_opcodes = df_opcodes.iloc[:,1:]
df_opcodes.head()

,label,OPCODE-rem-float/2addr,OPCODE-rem-int/2addr,OPCODE-array-length,OPCODE-div-int/lit8,OPCODE-sub-float/2addr,OPCODE-and-int/lit8,OPCODE-and-int,OPCODE-int-to-long,OPCODE-move-wide/from16,...,OPCODE-new-instance,OPCODE-packed-switch,OPCODE-mul-int,OPCODE-shl-int,OPCODE-ushr-long/2addr,OPCODE-iget-object,OPCODE-or-int/lit16,OPCODE-const-wide,OPCODE-iput-wide,OPCODE-or-int/2addr
0,BW,3,4,674,61,38,109,10,94,45,...,7393,289,13,8,10,49439,4,45,284,1562
1,BW,2,1,143,84,39,164,26,28,32,...,2097,94,17,15,1,9363,5,28,88,64
2,BW,2,1,143,84,39,164,26,28,32,...,2097,94,17,15,1,9363,5,28,88,64
3,BW,1,0,546,33,20,73,9,40,39,...,5292,238,11,7,6,45389,4,34,255,1550
4,MW,0,0,43,6,2,11,0,9,32,...,852,7,0,0,0,1076,0,0,4,0


## 2.2. Permissions 

In [4]:
#load_data
df_permissions = pd.read_csv("../data/interim/permissions.csv", )

In [5]:
df_permissions = df_permissions.iloc[:,1:]
df_permissions.head()

,label,PERMISSION-com.amazon.CONTENT_PROVIDER_ACCESS,PERMISSION-com.mducrad.taskmanagerapp.permission.C2D_MESSAGE,PERMISSION-com.mywavia.poker.android.permission.C2D_MESSAGE,PERMISSION-android.permission.INTERNET,PERMISSION-com.yandex.promolib.permission.ACCESS_START_SVC,PERMISSION-com.android.buycott.permission.C2D_MESSAGE,PERMISSION-com.triactivemedia.familytree.permission.C2D_MESSAGE,PERMISSION-air.hdandv.hashgab.permission.C2D_MESSAGE,PERMISSION-com.Settings.widget.permission.SettingsAppWidgetProvider,...,PERMISSION-com.android.vending.INSTALL_REFERRER,PERMISSION-com.sarkarinaukrijobs.permission.C2D_MESSAGE,PERMISSION-com.apus.hola.launcher.permission.READ_SETTINGS,PERMISSION-com.kddi.market.permission.USE_ALML,PERMISSION-com.vivek.ayyada.permission.C2D_MESSAGE,PERMISSION-appsw7x.cristiantg.buspucelapro.permission.MAPS_RECEIVE,PERMISSION-com.triactivemedia.otaku.permission.C2D_MESSAGE,PERMISSION-com.uob.makeourmark.permission.C2D_MESSAGE,PERMISSION-com.mode.mwalimukenya.ui.permission.MAPS_RECEIVE,PERMISSION-com.protonge.packs.permission.C2D_MESSAGE
0,BW,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,BW,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,BW,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,BW,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,MW,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## 2.3. System commands

In [6]:
#load_data
df_system = pd.read_csv("../data/interim/system_commands.csv")

In [7]:
df_system = df_system.iloc[:,1:]
df_system.head()

,label,SYSTEMCOMMAND-vold,SYSTEMCOMMAND-getevent,SYSTEMCOMMAND-setprop,SYSTEMCOMMAND-bugreport,SYSTEMCOMMAND-debuggerd,SYSTEMCOMMAND-sdcard,SYSTEMCOMMAND-chmod,SYSTEMCOMMAND-cd,SYSTEMCOMMAND-app_process,...,SYSTEMCOMMAND-svc,SYSTEMCOMMAND-run-as,SYSTEMCOMMAND-stagefright,SYSTEMCOMMAND-chown,SYSTEMCOMMAND-sh,SYSTEMCOMMAND-mediaserver,SYSTEMCOMMAND-gzip,SYSTEMCOMMAND-dexopt,SYSTEMCOMMAND-lsof,SYSTEMCOMMAND-nandread
0,BW,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,BW,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,BW,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,BW,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,MW,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,0,0


## 2.4. Receivers

In [8]:
# load data
df_receivers = pd.read_csv("../data/interim/intent_receivers.csv")
df_receivers = df_receivers.iloc[:,1:]

In [9]:
df_receivers.head()

,label,RECEIVER-,RECEIVER-it.samira.mylib.receivers.AudioPlayerReceiver_CONTROL,RECEIVER-com.igexin.sdk.action.cIRZ7PvDXY8zqOgEZSBGm1,RECEIVER-com.mobisleapps.maybe.like.description,RECEIVER-com.sec.everglades.main.update_for_widget,RECEIVER-org.mozilla.webapp.TASK_REMOVED,RECEIVER-action_show_10km,RECEIVER-com.cn21.android.pushservice.action.MESSAGE,RECEIVER-com.ganji.android.action.START_ALARM,...,RECEIVER-com.android.vending.INSTALL_REFERRER,RECEIVER-com.jb.gosms.im.IM_NEW_MESSAGE,RECEIVER-com.garena.android.gpns.NOTIFICATION_RECEIVE,RECEIVER-com.mymoney.sms.action.GET_CACHE_DATA_REMIND,RECEIVER-com.colormar.colorWeatherxdfghjwfwfghj.appWidget.ACTION_SHOW_MORE,RECEIVER-com.syezon.action.ADJUST_ORDER,RECEIVER-cn.jpush.android.intent.UNREGISTRATION,RECEIVER-com.appbyme.app64872.forum.service.heart.beat.notify,RECEIVER-com.optaim.zyzservice.SETUP,RECEIVER-android.intent.action.SEND_MESSAGE
0,BW,1,0,0,0,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,BW,1,0,0,0,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,BW,1,0,0,0,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,BW,1,0,0,0,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,MW,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
#split dataframe to benign apks and malware apks
df_bw_receivers = df_receivers[df_receivers["label"] == ''"BW"]
df_mw_receivers = df_receivers[df_receivers["label"] == "MW"]

## 2.5. Services

In [10]:
#load data
df_services = pd.read_csv("../data/interim/intent_services.csv")
df_services = df_services.iloc[:, 1:]

In [11]:
df_services.head()

,label,SERVICE-,SERVICE-com.editorinc.greetingcards.getwellsoon.modules.podcast.download.ACTION_STOP_DOWNLOADS,SERVICE-com.guide.flippywheelsguide.modules.podcast.player.ACTION_TOGGLE_PLAY,SERVICE-com.skb.btvmobile.popuplayservice.UI_SHOW_RESIZEBTN,SERVICE-com.borya.cclingtoneconcall.action.START_CCS_SERVICE,SERVICE-cn.mchina.haokan.action.user_sign_in,SERVICE-android.service.dreams.DreamService,SERVICE-de.bright_side.brightsound.service.action.NEXT,SERVICE-com.baidu.android.pushservice.action.PUSH_SERVICE,...,SERVICE-com.andromo.dev64620.app254035.soundboard.action.SKIP,SERVICE-com.juanjo.frasesparaenamorarmujer.modules.podcast.player.ACTION_STOP,SERVICE-mw.ocgvfhdkbxbb.lqqudspk.m42c1f214.wb190f26e.NEEVLLAZBHGWL,SERVICE-CLICK_CHIZROID,SERVICE-android.intent.action.CDE.HDP.MOBILE,SERVICE-uk.co.kavcom.zss_start,SERVICE-com.baihe.intent.STOP_PUSH_SERVICE,SERVICE-com.dgsd.ddg.modules.podcast.download.ACTION_STOP_DOWNLOADS,SERVICE-com.vblast.xiialive.intent.action.PLAYBACK_START,SERVICE-musicalatina.com.modules.podcast.player.ACTION_PAUSE
0,BW,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,BW,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,BW,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,BW,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,MW,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## 2.6. Activities

In [12]:
#load data
df_activities = pd.read_csv("../data/interim/intent_activities.csv")
df_activities = df_activities.iloc[:, 1:]

In [59]:
df_activities.head()

,label,ACTIVITY-,ACTIVITY-com.hanmiOnlinePharm.sfa.ui.threecall.VisitCallProgressActivity,ACTIVITY-android.intent.action.mamafu.INTERMEDIATE,ACTIVITY-com.fusepowered.fuseactivities.FuseApiMoregamesBrowser,ACTIVITY-io.rong.voipkit.calledSideActivity.action,ACTIVITY-android.intent.action.Run,ACTIVITY-com.quizmine.androidnutrition.tutorialActivity_slab,ACTIVITY-seven.client.ntr.android.QR.SCAN,ACTIVITY-com.experlabs.timepass.PREFS,...,ACTIVITY-com.sonyericsson.extras.liveware.SHOW_CUSTOMIZED_MSG,ACTIVITY-com.myingzhijia.CustomApplyServiceActivity,ACTIVITY-k.microcells.running.GameActivity,ACTIVITY-com.quizmine.thirdrdgradeccmath.howToUseVideoActivity,ACTIVITY-com.meituan.android.intent.action.address_edit_meituan_activity,ACTIVITY-android.intent.action.cpsl.checkoutuld,ACTIVITY-action.cidian.search,ACTIVITY-com.kukool.iphonestock.activity.AddCity,ACTIVITY-mituo.plat.intent.action.ADS.NOTIFIER.CLICKED,ACTIVITY-com.fresvii.chickenballoon.NotificationAction
0,BW,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,BW,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,BW,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,BW,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,MW,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## 2.7. API calls

In [14]:
#load data
df_api_calls = pd.read_csv("../data/interim/api_calls.csv")
df_api_calls = df_api_calls.iloc[:,1:]
df_api_calls.head()
df_api_calls.describe()

,APICALL-java.security.cert.Certificate,APICALL-android.view.SubMenu,APICALL-android.media.audiofx.Virtualizer,APICALL-android.support.v7.widget.helper.ItemTouchHelper,APICALL-android.net.RouteInfo,APICALL-android.bluetooth.BluetoothGattCallback,APICALL-android.support.percent.PercentLayoutHelper,APICALL-android.print.PageRange,APICALL-java.net.InterfaceAddress,APICALL-android.support.v7.widget.DividerItemDecoration,...,APICALL-android.speech,APICALL-android.content.ClipDescription,APICALL-android.support.v4.view.accessibility.AccessibilityManagerCompat,APICALL-java.net.InetAddress,APICALL-android.support.multidex.MultiDexApplication,APICALL-android.widget.MediaController,APICALL-android.support.v4.widget.ExploreByTouchHelper,APICALL-android.content.ContentResolver,APICALL-org.xmlpull.v1.XmlPullParser,APICALL-org.xml.sax.ext.DeclHandler
count,22000.000000,22000.000000,22000.000000,22000.000000,22000.000000,22000.000000,22000.000000,22000.00000,22000.000000,22000.000000,...,22000.0,22000.000000,22000.000000,22000.000000,22000.000000,22000.000000,22000.000000,22000.000000,22000.000000,22000.000000
mean,0.292545,2.596636,0.005591,5.097636,0.002636,0.003182,0.030364,0.00200,0.008500,0.001227,...,0.0,0.635682,1.367000,3.766773,0.025727,1.629273,12.803818,12.108273,31.229045,0.036455
std,2.113503,4.354389,0.230056,21.344695,0.179884,0.151927,0.601365,0.13617,0.222124,0.060667,...,0.0,0.950356,1.499259,8.454665,0.230167,4.756713,15.146042,25.144656,92.923261,0.383697
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.0,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,7.000000,9.000000,0.000000
75%,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.0,2.000000,3.000000,4.000000,0.000000,1.000000,30.000000,14.000000,16.000000,0.000000
max,50.000000,58.000000,16.000000,101.000000,13.000000,12.000000,12.000000,18.00000,19.000000,3.000000,...,0.0,15.000000,65.000000,212.000000,5.000000,102.000000,62.000000,1015.000000,2014.000000,10.000000


# TODO: 3. Feature Engineering

# 4. Preprocessing data

In [8]:
#feature_list = [df_activities, df_api_calls, df_opcodes, df_permissions, df_receivers, df_services, df_system]
feature_names = ["activities", "api_calls", "opcodes", "permissions", "receivers", "services", "system"]


In [3]:
print(feature_names[0])

activities


### Save preprocessed data

In [27]:
for i, feature in enumerate(tqdm(feature_list)):
    df_train, df_test = train_test_split(feature, test_size=None, train_size=0.7, random_state = 3)

    df_train = pd.get_dummies(df_train, columns=["label"], drop_first=True, prefix='', prefix_sep='')
    df_test = pd.get_dummies(df_test, columns=["label"], drop_first=True, prefix='', prefix_sep='')
    
    print("Saving "+feature_names[i]+" feature ...")
    df_train_sc.to_csv('../data/processed/test/train_'+ feature_names[i]+'_sc.csv', index=False)
    df_test_sc.to_csv('../data/processed/test/test_'+ feature_names[i]+'_sc.csv', index=False)

    df_train.to_csv("../data/processed/test/train_"+ feature_names[i]+".csv", index=False)
    df_test.to_csv("../data/processed/test/test_"+ feature_names[i]+".csv", index=False)
    print(feature_names[i] +" saved.")
    

Saving activities feature ...
activities saved.
Saving api_calls feature ...
api_calls saved.
Saving opcodes feature ...
opcodes saved.
Saving permissions feature ...
permissions saved.
Saving receivers feature ...
receivers saved.
Saving services feature ...
services saved.
Saving system feature ...
system saved.


### Load preprocessed data

In [9]:
def load_feature(feature):
    
    print("Loading "+feature +" features ...")
    train_df = pd.read_csv("../data/processed/train_"+ feature+".csv")
    test_df = pd.read_csv("../data/processed/test_"+ feature+".csv")
    print(feature +" loaded.")
    
    return train_df, test_df

In [3]:
df_train[0]

NameError: name 'df_train' is not defined

In [10]:
df_train = df_test = df_train_sc = df_test_sc = [None]*len(feature_names)
X = y = X_sc = y_sc = X_test = X_test_sc = [None]*len(feature_names)

In [11]:
for i, feature in enumerate(tqdm(feature_names)):
    df_train[i], df_test[i] = load_feature(feature)
    df_1 = df_train[i] # Cannot use df_train[i].drop() -> "MW not found" in axis
    df_2 = df_test[i]
    scaler = StandardScaler()
    scaler.fit(df_1.drop(['MW'], axis=1))
    scaled_features = scaler.transform(df_1.drop(['MW'],axis=1))
    df_train_sc[i] = pd.DataFrame(scaled_features) # columns=df_train_ml.columns[1::])

    scaled_features = scaler.transform(df_2.drop(['MW'],axis=1))
    df_test_sc[i] = pd.DataFrame(scaled_features) # , columns=df_test_ml.columns)

    X[i] = df_1.drop('MW', axis=1)
    y[i] = df_1["MW"]
    X_test[i] = df_2

    X_sc[i] = df_train_sc[i]
    y_sc[i] = df_1['MW']
    X_test_sc[i] = df_test_sc[i]

  0%|          | 0/7 [00:00<?, ?it/s]

Loading activities features ...
activities loaded.


 14%|█▍        | 1/7 [00:51<05:09, 51.65s/it]

Loading api_calls features ...
api_calls loaded.


 29%|██▊       | 2/7 [01:02<03:16, 39.39s/it]

...

Loading services features ...
services loaded.


 86%|████████▌ | 6/7 [03:19<00:38, 38.56s/it]

Loading system features ...


100%|██████████| 7/7 [03:20<00:00, 28.57s/it]

system loaded.


In [13]:
X_test[0]

,0,1,2,3,4,5,6,7,8,9,...,6079,6080,6081,6082,6083,6084,6085,6086,6087,6088
0,0.0,0.0,0.0,-0.021323,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,-0.021323,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,-0.021323,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,-0.021323,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,-0.021323,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6596,0.0,0.0,0.0,-0.021323,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6597,0.0,0.0,0.0,-0.021323,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6598,0.0,0.0,0.0,-0.021323,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6599,0.0,0.0,0.0,-0.021323,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
X[0]

,0,1,2,3,4,5,6,7,8,9,...,6079,6080,6081,6082,6083,6084,6085,6086,6087,6088
0,0.0,0.0,0.0,-0.021323,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,-0.021323,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,-0.021323,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,-0.021323,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,-0.021323,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6596,0.0,0.0,0.0,-0.021323,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6597,0.0,0.0,0.0,-0.021323,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6598,0.0,0.0,0.0,-0.021323,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6599,0.0,0.0,0.0,-0.021323,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Train models for each feature

In [93]:
gsRFC = [None]*len(feature_names)
RFC_best = [None]*len(feature_names)

for i in range(len(X_sc)):
    cross_validation_split = model_selection.ShuffleSplit(n_splits = 10, test_size = .3, train_size = .7, random_state = 0 )

    # RFC Parameters tunning 

    RFC = RandomForestClassifier()


    ## Search grid for optimal parameters
    rf_param_grid = {"max_depth": [None],
                  "max_features": [1, 3, 10],
                  "min_samples_split": [2, 3, 10],
                  "min_samples_leaf": [1, 3, 10],
                  "bootstrap": [False],
                  "n_estimators" :[100,300],
                  "criterion": ["gini"]}



    gsRFC[i] = GridSearchCV(RFC,param_grid = rf_param_grid, cv=cross_validation_split, scoring="accuracy", n_jobs= 12, verbose = 10)

    gsRFC[i].fit(X_sc[i],df_1['MW'])

    RFC_best[i] = gsRFC[i].best_estimator_

    # Best score
    gsRFC[i].best_score_

Fitting 10 folds for each of 54 candidates, totalling 540 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:   24.7s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:   46.8s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:  1.6min
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:  2.4min
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:  3.6min
[Parallel(n_jobs=8)]: Done  45 tasks      | elapsed:  4.3min
[Parallel(n_jobs=8)]: Done  56 tasks      | elapsed:  5.5min
[Parallel(n_jobs=8)]: Done  69 tasks      | elapsed:  5.6min
[Parallel(n_jobs=8)]: Done  82 tasks      | elapsed:  5.7min
[Parallel(n_jobs=8)]: Done  97 tasks      | elapsed:  5.8min
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:  5.8min
[Parallel(n_jobs=8)]: Done 129 tasks      | elapsed:  5.9min
[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:  6.0min
[Parallel(n_jobs=8)]: Done 165 tasks      | elapsed:  6.1min
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:  6.6min
[Parallel(

Fitting 10 folds for each of 54 candidates, totalling 540 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    4.6s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    7.4s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:   13.3s
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:   18.2s
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:   25.4s
[Parallel(n_jobs=8)]: Done  45 tasks      | elapsed:   29.6s
[Parallel(n_jobs=8)]: Done  56 tasks      | elapsed:   36.1s
[Parallel(n_jobs=8)]: Done  69 tasks      | elapsed:   38.8s
[Parallel(n_jobs=8)]: Done  82 tasks      | elapsed:   41.0s
[Parallel(n_jobs=8)]: Done  97 tasks      | elapsed:   43.4s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:   45.8s
[Parallel(n_jobs=8)]: Done 129 tasks      | elapsed:   47.4s
[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:   49.6s
[Parallel(n_jobs=8)]: Done 165 tasks      | elapsed:   52.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:   56.5s
[Parallel(

Fitting 10 folds for each of 54 candidates, totalling 540 fits


...

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:   28.3s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:   54.9s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:  1.8min
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:  2.6min
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:  3.8min
[Parallel(n_jobs=8)]: Done  45 tasks      | elapsed:  4.3min
[Parallel(n_jobs=8)]: Done  56 tasks      | elapsed:  5.1min
[Parallel(n_jobs=8)]: Done  69 tasks      | elapsed:  5.4min
[Parallel(n_jobs=8)]: Done  82 tasks      | elapsed:  5.6min
[Parallel(n_jobs=8)]: Done  97 tasks      | elapsed:  5.8min
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:  5.9min
[Parallel(n_jobs=8)]: Done 129 tasks      | elapsed:  6.0min
[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:  6.1min
[Parallel(n_jobs=8)]: Done 165 tasks      | elapsed:  6.3min
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:  6.8min
[Parallel(

Fitting 10 folds for each of 54 candidates, totalling 540 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:   20.2s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:   36.8s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:  1.3min
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:  1.8min
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:  2.7min
[Parallel(n_jobs=8)]: Done  45 tasks      | elapsed:  3.3min
[Parallel(n_jobs=8)]: Done  56 tasks      | elapsed:  4.2min
[Parallel(n_jobs=8)]: Done  69 tasks      | elapsed:  4.3min
[Parallel(n_jobs=8)]: Done  82 tasks      | elapsed:  4.3min
[Parallel(n_jobs=8)]: Done  97 tasks      | elapsed:  4.4min
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:  4.5min
[Parallel(n_jobs=8)]: Done 129 tasks      | elapsed:  4.5min
[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:  4.6min
[Parallel(n_jobs=8)]: Done 165 tasks      | elapsed:  4.6min
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:  5.0min
[Parallel(

Fitting 10 folds for each of 54 candidates, totalling 540 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    3.3s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    4.9s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    8.3s
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:   11.2s
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:   15.6s
[Parallel(n_jobs=8)]: Done  45 tasks      | elapsed:   18.3s
[Parallel(n_jobs=8)]: Done  56 tasks      | elapsed:   22.4s
[Parallel(n_jobs=8)]: Done  69 tasks      | elapsed:   24.1s
[Parallel(n_jobs=8)]: Done  82 tasks      | elapsed:   25.5s
[Parallel(n_jobs=8)]: Done  97 tasks      | elapsed:   27.0s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:   28.3s
[Parallel(n_jobs=8)]: Done 129 tasks      | elapsed:   29.3s
[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:   30.6s
[Parallel(n_jobs=8)]: Done 165 tasks      | elapsed:   32.1s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:   34.7s
[Parallel(

In [ ]:
gsLGB = [None]*len(feature_names)
gsLGB_best = [None]*len(feature_names)

for i in range(len(X)):
    cross_validation_split = model_selection.ShuffleSplit(n_splits = 10, test_size = .3, train_size = .7, random_state = 0 )
    n_estimators=list(range(100,1100,100))
    learn_rate=[0.01,0.05,0.1,0.2,0.3,0.25,0.4,0.5,0.6,0.7,0.8,0.9,1]
    print(feature_names[i])
    
    hyper={'n_estimators':n_estimators,'learning_rate':learn_rate}
    gsLGB[i]=GridSearchCV(estimator=lgb.LGBMClassifier(),param_grid=hyper,verbose=True,n_jobs = 12, cv=cross_validation_split)
    gsLGB[i].fit(X[i],df_1['MW'])
    
    gsLGB_best[i] = gsLGB[i].best_estimator_
    print(gsLGB[i].best_score_)

In [67]:
gsKnearest = [None]*len(feature_names)
gsKnearest_best = [None]*len(feature_names)

for i in range(len(X)):
    cross_validation_split = model_selection.ShuffleSplit(n_splits = 10, test_size = .3, train_size = .7, random_state = 0 )
    n_neighbors=list(range(1,10,1))
    weights=["uniform","distance"]
    
    print(feature_names[i])
    
    hyper={'n_neighbors':n_neighbors,'weights':weights}
    gsKnearest[i]=GridSearchCV(estimator=KNeighborsClassifier(),param_grid=hyper,verbose=True,n_jobs = 12, cv=cross_validation_split)
    gsKnearest[i].fit(X[i],df_1['MW'])
    
    gsKnearest_best[i] = gsKnearest[i].best_estimator_
    print(gsKnearest[i].best_score_)

activities
Fitting 10 folds for each of 18 candidates, totalling 180 fits


[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed: 15.4min
[Parallel(n_jobs=12)]: Done 180 out of 180 | elapsed: 78.5min finished


0.504946996466431
api_calls
Fitting 10 folds for each of 18 candidates, totalling 180 fits


[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:  4.0min
[Parallel(n_jobs=12)]: Done 180 out of 180 | elapsed: 22.3min finished


0.846643109540636
opcodes
Fitting 10 folds for each of 18 candidates, totalling 180 fits


...

[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed: 11.6min
[Parallel(n_jobs=12)]: Done 180 out of 180 | elapsed: 57.7min finished


0.5115598182735992
system
Fitting 10 folds for each of 18 candidates, totalling 180 fits


[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:   16.5s


0.7742554265522463


[Parallel(n_jobs=12)]: Done 180 out of 180 | elapsed:  1.3min finished


In [ ]:
gs_ADA = [None]*len(feature_names)
gs_ADA_best = [None]*len(feature_names)

for i in range(len(X)):
    cross_validation_split = model_selection.ShuffleSplit(n_splits = 10, test_size = .3, train_size = .7, random_state = 0 )
    n_estimators=list(range(100,1100,100))
    learn_rate=[0.01,0.05,0.1,0.2,0.3,0.25,0.4,0.5,0.6,0.7,0.8,0.9,1]
    hyper={'n_estimators':n_estimators,'learning_rate':learn_rate}
    
    print(feature_names[i])
    
    gs_ADA[i]=RandomizedSearchCV(estimator=AdaBoostClassifier(),n_iter=10,param_distributions=hyper,verbose=True, n_jobs=12, cv=cross_validation_split)
    gs_ADA[i].fit(X[i],df_1["MW"])
    
    gs_ADA_best[i]=gs_ADA[i].best_score_
    print(gs_ADA[i].best_score_)

In [62]:
gs_xgb = [None]*len(feature_names)
gs_xgb_best = [None]*len(feature_names)

for i in range(len(X)):
    cross_validation_split = model_selection.ShuffleSplit(n_splits = 10, test_size = .3, train_size = .7, random_state = 0 )
    n_estimators=list(range(100,1100,100))
    learn_rate=[0.01,0.05,0.1,0.2,0.3,0.25,0.4,0.5,0.6,0.7,0.8,0.9,1]
    hyper={'n_estimators':n_estimators,'learning_rate':learn_rate}
    
    print(feature_names[i])

#     params = {  
#            'tree_method': 'gpu_hist', 
#            'gpu_id': 0,            
#            "predictor":'gpu_predictor',
#            } 
    
    gs_xgb[i]=RandomizedSearchCV(estimator=xgb.XGBClassifier(),param_distributions=hyper,n_iter=10,verbose=True, cv=cross_validation_split, n_jobs=12)
    gs_xgb[i].fit(X[i],df_1["MW"])
    
    gs_xgb_best[i]=gs_xgb[i].best_score_
    print(gs_xgb[i].best_score_)

activities
Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed: 18.2min
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed: 46.6min finished


0.5098939929328622
api_calls
Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed: 12.9min
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed: 31.5min finished


0.8825845532559313
opcodes
Fitting 10 folds for each of 10 candidates, totalling 100 fits


...

0.8493690055527512
services
Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed: 13.2min
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed: 47.4min finished


0.509439676930843
system
Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:   31.3s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:  1.7min finished


0.7890964159515397


In [20]:
gsLGB[1].best_estimator_

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.05, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=500, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [1]:
# Save model
for i, n in enumerate(feature_names):
    filename = 'gsKnearest_'+ n + '_model.sav'
    pickle.dump(gs_Knearest[i], open("../models/Knearest/"+filename, 'wb')) 

NameError: name 'feature_names' is not defined

In [3]:
%time df_train = pd.read_csv("../data/processed/test/train.csv")
%time df_test  = pd.read_csv("../data/processed/test/test.csv")

df_train.head(1)

CPU times: user 2min 15s, sys: 3.79 s, total: 2min 18s
Wall time: 2min 19s
CPU times: user 1min 1s, sys: 996 ms, total: 1min 2s
Wall time: 1min 25s


,PERMISSION-com.amazon.CONTENT_PROVIDER_ACCESS,PERMISSION-com.mducrad.taskmanagerapp.permission.C2D_MESSAGE,PERMISSION-com.mywavia.poker.android.permission.C2D_MESSAGE,PERMISSION-android.permission.INTERNET,PERMISSION-com.yandex.promolib.permission.ACCESS_START_SVC,PERMISSION-com.android.buycott.permission.C2D_MESSAGE,PERMISSION-com.triactivemedia.familytree.permission.C2D_MESSAGE,PERMISSION-air.hdandv.hashgab.permission.C2D_MESSAGE,PERMISSION-com.Settings.widget.permission.SettingsAppWidgetProvider,PERMISSION-com.ionicframework.codeartfilmnews445890.permission.C2D_MESSAGE,...,RECEIVER-com.jb.gosms.im.IM_NEW_MESSAGE,RECEIVER-com.garena.android.gpns.NOTIFICATION_RECEIVE,RECEIVER-com.mymoney.sms.action.GET_CACHE_DATA_REMIND,RECEIVER-com.colormar.colorWeatherxdfghjwfwfghj.appWidget.ACTION_SHOW_MORE,RECEIVER-com.syezon.action.ADJUST_ORDER,RECEIVER-cn.jpush.android.intent.UNREGISTRATION,RECEIVER-com.appbyme.app64872.forum.service.heart.beat.notify,RECEIVER-com.optaim.zyzservice.SETUP,RECEIVER-android.intent.action.SEND_MESSAGE,MW
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df_train_sc = pd.read_csv("../data/processed/test/train_sc.csv")
df_test_sc = pd.read_csv("../data/processed/test/test_sc.csv")

df_train_sc.head(2)

,0,1,2,3,4,5,6,7,8,9,...,24815,24816,24817,24818,24819,24820,24821,24822,24823,24824
0,0.0,0.0,0.0,0.219846,0.0,0.0,0.0,0.0,-0.012742,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.219846,0.0,0.0,0.0,0.0,-0.012742,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Load models

In [69]:
lgbm_models = []
rfc_models = []
knearest_models = []

for i, n in enumerate(feature_names):
    filename = 'gsLGB_'+ n + '_model.sav'
    lgbm_models.append(pickle.load(open("../models/LightGBM/"+filename, 'rb')))
    
for i, n in enumerate(feature_names):
    filename = 'gsRFC_'+ n + '_model.sav'
    rfc_models.append(pickle.load(open("../models/Random_Forest_Classifier/"+filename, 'rb')))

for i, n in enumerate(feature_names):
    filename = 'gsKnearest_'+ n + '_model.sav'
    knearest_models.append(pickle.load(open("../models/Knearest/"+filename, 'rb')))

for i, n in enumerate(feature_names):
    filename = 'gsXGBM_'+ n + '_model.sav'
    knearest_models.append(pickle.load(open("../models/xgbm/"+filename, 'rb')))

In [43]:
for i, model in enumerate(lgbm_models):
    cv_results = model.cv_results_
    cv_mean = cv_results["mean_test_score"].mean()
    result = model.score(X_test_sc[i], df_2["MW"])
    print(feature_names[i] + "  cv_result : "  + str(cv_mean))
    print(feature_names[i] +"  result on test data: " + str(result))

activities  cv_result : 0.509439676930843
activities  result on test data: 0.509619754582639
api_calls  cv_result : 0.8782856366248593
api_calls  result on test data: 0.99409180427208
opcodes  cv_result : 0.8518875470818933
opcodes  result on test data: 0.9954552340554461
permissions  cv_result : 0.7978526773579776
permissions  result on test data: 0.8947129222845024
receivers  cv_result : 0.8485900671766397
receivers  result on test data: 0.9778821390698379
services  cv_result : 0.509439676930843
services  result on test data: 0.509619754582639
system  cv_result : 0.7866019492874617
system  result on test data: 0.8992576882290562


In [72]:
for i, model in enumerate(knearest_models):
    cv_mean = cv_results["mean_test_score"].mean()
    result = model.score(X_test_sc[i], df_2["MW"])
    print(feature_names[i] + "  cv_result : "  + str(cv_mean))
    print(feature_names[i] +"  result on test data: " + str(result))

activities  cv_result : 0.5098939929328622
activities  result on test data: 0.5105287077715498
api_calls  cv_result : 0.8740232205956587
api_calls  result on test data: 0.9903044993182851
opcodes  cv_result : 0.8469005552751137
opcodes  result on test data: 0.9959097106499015
permissions  cv_result : 0.8008228167592126
permissions  result on test data: 0.8842599606120285
receivers  cv_result : 0.8387985865724381
receivers  result on test data: 0.9139524314497803
services  cv_result : 0.5094396769308431
services  result on test data: 0.509619754582639
system  cv_result : 0.7808531044926805
system  result on test data: 0.8918345705196182


In [42]:
for i, model in enumerate(rfc_models):
    cv_results = model.cv_results_
    cv_mean = cv_results["mean_test_score"].mean()
    result = model.score(X_test_sc[i], df_2["MW"])
    print(feature_names[i] + "  cv_result : "  + str(cv_mean))
    print(feature_names[i] +"  result on test data: " + str(result))

activities  cv_result : 0.5098126647596611
activities  result on test data: 0.515527950310559
api_calls  cv_result : 0.8238011105502273
api_calls  result on test data: 0.994849265262839
opcodes  cv_result : 0.834831828294726
opcodes  result on test data: 0.9959097106499015
permissions  cv_result : 0.6143296501953744
permissions  result on test data: 0.9292531434631116
receivers  cv_result : 0.7946192532764969
receivers  result on test data: 0.9846992879866687
services  cv_result : 0.5064109035840484
services  result on test data: 0.509619754582639
system  cv_result : 0.752608110381962
system  result on test data: 0.926829268292683


In [52]:
from sklearn.ensemble import VotingClassifier


votingC = VotingClassifier(estimators=[('rfc_api_calls', rfc_models[1].best_estimator_), ('lgbm_api_calls', lgbm_models[1].best_estimator_)], voting='soft', n_jobs=8)

%%time votingC = votingC.fit(X_sc[1], df_1["MW"])

In [56]:
votingC.score(X_test_sc[1], df_2["MW"])

0.994849265262839